In [1]:
import numpy as np   
import pandas as pd        
from scipy import stats as sts 

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

from math import factorial as fact

## Задачи на проверку гипотез

### Задача №1: Джеймс Бонд (доля + 1 выборка)

<img src="../images/bond.png" width="300" align='left'>

Ваш друг Джеймс Бонд утверждает, что умеет отличать взболтанный мартини от смешанного. Вы решили проверить это. Чтобы сделать это, вы завязали глаза Бонду и дали попробовать ему мартини 10 раз. Пусть Бонд отгадал тип мартини 6 раз и не угадал 4. Давайте проверим гипотезу о том, что Бонд умеет отличать мартини, используя z-test.

*Выборка:* $1, 0, 0, 1, 1, 0, 0, 1, 1, 1$

$$
\begin{aligned}
& H_0: p = 0.5 \text{(не отличает мартини)} \\
& H_1: p > 0.5 \text{(отличает мартини)}
\end{aligned}
$$


$$
X_1, \ldots, X_n \sim Bern(p)
$$

$$
\begin{aligned}
& E(X) = p \\
& Var(X) = p \cdot (1-p)
\end{aligned}
$$

In [17]:
X = np.array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1])
n = len(X)
alpha = 0.05

1. Давайте проверим вручную

In [22]:
p_0 = 0.5
var_0 = p_0 * (1 - p_0)
# var_0 = np.var(X)


p_hat = np.mean(X)
print(f'Выборочная доля: {p_hat}')


z_sample = (p_hat - p_0) / np.sqrt(var_0 / n)
print(f'Выборочное z: {z_sample}')


norm_gen = sts.norm(0, 1)
z_crit = norm_gen.ppf(1 - alpha)
print(f'Критическое z: {z_crit}')

# делаем вывод:
print()
if z_sample > z_crit:
    print('Нулевая гипотеза отвергается')
else:
    print('Нулевая гипотеза не отвергается')

Выборочная доля: 0.6
Выборочное z: 0.6324555320336757
Критическое z: 1.6448536269514722

Нулевая гипотеза не отвергается


2. Проверим одной функцией

In [19]:
from statsmodels.stats.proportion import proportions_ztest

In [23]:
proportions_ztest(
    sum(X),              # кол-во единиц
    len(X),              # n 
    value=0.5,           # p_0
    alternative='larger' # альтернативная гипотеза (larger / smaller / two-sided)
)

(0.6454972243679027, 0.25930250821436285)

### Задача №2: О мышах и людях

<img src="../images/vagon.jpeg" width="500" align='left'>

Для изучения аспектов процесса принятия моральных решений психологи уже много лет используют этические дилеммы, с помощью которых оценивают действия людей в гипотетических ситуациях. Один из самых известных примеров — это [проблема вагонетки,](https://ru.wikipedia.org/wiki/Проблема_вагонетки) в которой необходимо принять решение о том, стоит ли пожертвовать одним человеком для спасения пятерых.

Бельгийские психологи воплотили дилемму в реальную жизнь. Участники эксперимента должны были выбрать, ударить током пять мышей или одну мышь. Эксперимент проходил следующим образом. Участника сажали перед двумя клетками, в одной из которых сидели пять мышей, а в другой — одна. Перед клетками стоял ноутбук с $20$-секундным таймером: участникам сообщили, что по истечении этих $20$ секунд в клетку с пятью мышами пустят ток, и мыши получат не смертельный, но очень болезненный удар. Пока время идет, участник может нажать на кнопку: в этом случае ток пустят по клетке с одной мышью. В исследовании использовали живых мышей. 

Удары тока были ненастоящими: сразу же после «удара» участников сопроводили в отдельную комнату, где разъяснили им, что мыши в полном порядке и током их не били (об этом заранее догадались только $12$ участников). В решении реальной проблемы вагонетки приняли участие $192$ человека, а еще $83$ решали такую же задачку, но гипотетическую (на бумаге). Все участники также прошли онлайн-опросы, в ходе которых учёные собрали о респондентах кучу дополнительной информации. 

В файле `mouse.csv` лежит информация о том, как прошёл эксперимент. Нас будут интересовать столбцы: 

* __STUDY:__ какую проблему вагонетки решал человек $1$, если на бумаге и $2$, если реальную
* __AGE:__ возраст респондента 
* __GENDER:__ пол респондента
* __DECISION:__ решение дилеммы ($1$ - жать на кнопу, $0$ - не жать) 
* __RT:__ время, которое респондент потратил, чтобы нажать на кнопку 

Подробное описание данных, сами данные и даже код на R, использованный при оценивании моделей, можно найти в [репозитории исследования.](https://osf.io/kvb99/) В статье авторы строили несколько логистических регрессий, чтобы очистить эффект от психологических особенностей респондентов. Про подобные приёмы очистки мы немного поговорим позже.  Также более подробно про исследование [можно почитать на N + 1.](https://nplus1.ru/news/2018/05/11/mice-trolley)

In [24]:
df = pd.read_csv('../data/mouse.csv', sep='\t')

# отбираем нужные колонки
df = df[['STUDY', 'AGE', 'GENDER', 'DECISION', 'RT']]

# удаляем пропуски по колонке DECISION
#     (то есть тех, кто не смог принять решение)
df = df[~df.DECISION.isnull()]

In [25]:
df.head()

,STUDY,AGE,GENDER,DECISION,RT
3,1,21,0,1.0,9.212427
4,1,20,0,1.0,17.071711
5,1,21,1,1.0,9.827884
6,1,23,1,1.0,10.143030
7,1,20,1,1.0,7.447534


### Гипотеза №1: Доля и нажатия на кнопку (доля + 2 выборки)

Попытаемся ответить на тот же вопрос, что и исследователи. В какой из двух ситуаций респонденты чаще принимают решение нажать кнопку: в реальной или на бумаге?

$$
\begin{aligned}
&H_0: \hspace{2mm} p_1 = p_2 \hspace{2mm} \text{(На бумаге и в реальности жмут на кнопку одинаково часто)} \\
&H_1: \hspace{2mm} p_1 > p_2  \hspace{2mm} \text{(На бумаге на кнопку жмут чаще)}
\end{aligned}
$$

In [32]:
X_paper = df[df['STUDY'] == 1].DECISION.values
X_real = df[df['STUDY'] == 2].DECISION.values


p_paper = np.mean(X_paper)
p_real = np.mean(X_real)

print(f'Доля нажатий на бумаге: {p_paper}')
print(f'Доля нажатий в реальных условиях: {p_real}')

Доля нажатий на бумаге: 0.8645833333333334
Доля нажатий в реальных условиях: 0.7469879518072289


ТАК ДЕЛАТЬ НЕЛЬЗЯ:

$$
0.86 > 0.74 \Rightarrow \text{ на бумаге жмут чаще}
$$

$$
\begin{aligned}
&H_0: \hspace{2mm} p_1 - p_2 = 0 \hspace{2mm} \text{(На бумаге и в реальности жмут на кнопку одинаково часто)} \\
&H_1: \hspace{2mm} p_1 - p_2 > 0 \hspace{2mm} \text{(На бумаге на кнопку жмут чаще)}
\end{aligned}
$$


$$
\begin{aligned}
&H_0: \hspace{2mm} p_{diff} = 0 \hspace{2mm} \text{(На бумаге и в реальности жмут на кнопку одинаково часто)} \\
&H_1: \hspace{2mm} p_{diff} > 0 \hspace{2mm} \text{(На бумаге на кнопку жмут чаще)}
\end{aligned}
$$

In [35]:
alpha = 0.05

In [36]:
p_diff_0 = 0

p_diff_sample = p_paper - p_real

# для двух выборок
P = (sum(X_paper) + sum(X_real)) / (len(X_paper) + len(X_real))

z_sample = (p_diff_sample - p_diff_0) / np.sqrt(P * (1-P) * (1 / len(X_paper) + 1 / len(X_real)))
print(f'Выборочное z: {z_sample}')


norm_gen = sts.norm(0, 1)
z_crit = norm_gen.ppf(1 - alpha)
print(f'Критическое z: {z_crit}')

# делаем вывод:
print()
if z_sample > z_crit:
    print('Нулевая гипотеза отвергается')
else:
    print('Нулевая гипотеза не отвергается')

Выборочное z: 2.3780989461645565
Критическое z: 1.6448536269514722

Нулевая гипотеза отвергается


In [37]:
proportions_ztest(
    (sum(X_paper), sum(X_real)),              # кол-во единиц
    (len(X_paper), len(X_real)),              # n 
    value=0,                                  # p_0
    alternative='larger'                      # альтернативная гипотеза (larger / smaller / two-sided)
)

(2.3780989461645565, 0.008701077805778048)

### Гипотеза №2. Среднее и кровожадность (среднее + 1 выборка)

Кровожадные люди быстро берут на себя ответственность за удар мышки током. Будем считать, что кровожадные люди принимают решение менее, чем за пять секунд. Правда ли, что люди по своей природе кровожадные? 

$$
\begin{aligned}
&H_0: \hspace{2mm} \mu \le 5 \hspace{2mm} \text{(Люди кровожадны)} \\
&H_1: \hspace{2mm} \mu > 5  \hspace{2mm} \text{(Люди не кровожадны)}
\end{aligned}
$$

In [45]:
X = df[~df['RT'].isnull()].RT.values

mu_hat = np.mean(X)
print(f'Выборочное среднее: {mu_hat}')


mu_0 = 5
alpha = 0.05
var_sample = np.var(X)
n = len(X)

z_sample = (mu_hat - mu_0) / np.sqrt(var_sample / n)
print(f'Выборочное z: {z_sample}')


norm_gen = sts.norm(0, 1)
z_crit = norm_gen.ppf(1 - alpha)
print(f'Критическое z: {z_crit}')


# делаем вывод:
print()
if z_sample > z_crit:
    print('Нулевая гипотеза отвергается')
else:
    print('Нулевая гипотеза не отвергается')

Выборочное среднее: 10.116762704819276
Выборочное z: 13.014154424812848
Критическое z: 1.6448536269514722

Нулевая гипотеза отвергается


## Делаем красиво

In [46]:
def my_test(X, mu_0, alpha):
    
    mu_hat = np.mean(X)
    var_hat = np.var(X)
    n = len(X)
    
    z_sample = (mu_hat - mu_0) / np.sqrt(var_hat / n)
    
    norm_gen = sts.norm(0, 1)
    z_crit = norm_gen.ppf(1 - alpha)
    
    if z_sample > z_crit:
        print('Нулевая гипотеза отвергается')
    else:
        print('Нулевая гипотеза не отвергается')

In [50]:
X = [1, 2, 4, 10, 24, 100]
mu_0 = 5
alpha = 0.01

In [51]:
my_test(
    X = X,
    mu_0 = mu_0,
    alpha = alpha
)

Нулевая гипотеза не отвергается
